In [ ]:
from pprint import pprint
import os
import sys

#name of the cluster
your_cluster_name = ""

## The private key pair for accessing cluster.
private_key = "/path/to/key.pem"

## If delete cfncluster after job is done./
delete_cfncluster= False

print("variables set")

In [ ]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager
from util import AddonsManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

### Check Scripts on Cluster

This section containss cells that can be used to check which scripts are currently on the cluster being used. 

#### Get all supported pipelines

In [ ]:
#This cell must be run before others in this section

scripts = AddonsManager.get_scripts_dict(ssh_client)
print()
print("Supported Pipelines:", AddonsManager.get_all_pipeline_names(scripts))

#### Check which workflows are in a given pipeline

In [ ]:
#can be set to a supported pipeline name or "all"
target_pipeline = "all"

print("Supported Workflows in {} Pipeline(s): ".format(target_pipeline), end="")
pprint(AddonsManager.get_workflows_in_pipeline(scripts, target_pipeline), indent=2)

#### Check which shell scripts are in a given pipeline or workflow

In [ ]:
#can be set to a supported pipeline or "all"
target_pipeline = "all"

#can be set to a support workflow or "all"
#if target_pipeline == "all" then this variable is ignored
target_workflow = "all"

pprint(AddonsManager.get_scripts(scripts, target_pipeline, target_workflow))

#### Check which step calls a script

In [ ]:
#should be the name of a shell script on the cluster (include sh extension)
target_script = ""

print(AddonsManager.get_steps_calling_script(ssh_client, scripts, target_script))

#### Print out a script

In [ ]:
#all targets must be set to a valid pipeline/workflow/script
#use cells above to check valid options
target_pipeline = ""
target_workflow = ""
target_script = ""


loc, file_cat = AddonsManager.cat_script(ssh_client, scripts, target_pipeline, target_workflow, target_script)
print("".join(["#"]*len(loc)) + "\n{}\n".format(loc) + "".join(["#"]*len(loc)))
print()
print(file_cat)

### Check Software on Cluster

This section is used to check what software is installed on the cluster.

#### Get all software installed on cluster

In [ ]:
software = AddonsManager.get_software_dict(ssh_client)
print("All Installed Software: ")
pprint(software)

#### Check the version of a specific tool

In [ ]:
#ignores case
target_tool = ""

print(AddonsManager.check_tool_is_installed(software, target_tool))